# Investigating Kilohertz (kHz) Frequency Stimulation Block
This tutorial will use the fiber model created in the [finite amplitudes tutorial](1_finite_amp.ipynb) to run a bisection search for the fiber's block threshold (i.e, the minimum stimulation amplitude needed to stop the progression of a propagating action potential).

## Create the fiber and set up simulation
As before, we create a fiber and electrical potentials

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.signal as sg
from pyfibers import build_fiber, FiberModel, ScaledStim

# create fiber model
n_sections = 265
fiber = build_fiber(FiberModel.MRG_INTERPOLATION, diameter=10, n_sections=n_sections)

# define unscaled extracellular potentials (in response to a unitary current)
fiber.potentials = fiber.point_source_potentials(0, 250, fiber.length / 2, 1, 10)
plt.plot(fiber.longitudinal_coordinates, fiber.potentials)
plt.xlabel('Distance along fiber (μm)')
plt.ylabel('Electrical potential (mV)')
plt.title('Extracellular potentials')

# turn on saving of transmembrane potential (Vm) so that we can plot it later
fiber.set_save_vm()

We'll create a 20 kHz square wave, using a longer time step than the previous tutorials so we have time to observe the fiber's behavior before, during, and after block. We will set the block signal to only be on from t=50 to t=100 ms. Then we'll add our waveform to a new ScaledStim instance.

In [ ]:
# Setup for simulation
time_step = 0.001  # ms
time_stop = 150  # ms
frequency = 20  # khz (because our time units are ms)

# create time vector to make waveform with sg
t = np.arange(0, time_stop, time_step)  # ms|
waveform = sg.square(2 * np.pi * frequency * t)

# blank out first and last thirds of simulation so we can see block turn on and off
waveform[t <= 50] = 0
waveform[t >= 100] = 0

# Create stimulation object
blockstim = ScaledStim(waveform=waveform, dt=time_step, tstop=time_stop)

Let's plot the whole waveform, as well as the point where it turns on and off.

In [ ]:
# whole waveform (cannot see individual pulses)
plt.plot(t, waveform)
plt.xlim([0, 150])
plt.xlabel('Time (ms)')
plt.ylabel('Unscaled Stimulation Amplitude')
plt.gcf().set_size_inches((10, 5))

# create figure to show kilohertz waveform details (turning on and off)
fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

# zoom in: khz turning on
ax[0].plot(t, waveform)
ax[0].set_xlim([49.8, 50.2])
ax[0].set_xlabel('Time (ms)\n\ntruncated x-limits to show waveform')
ax[0].set_ylabel('Unscaled Stimulation Amplitude')

# zoom in: khz turning off
ax[1].plot(t, waveform)
ax[1].set_xlim([99.8, 100.2])
ax[1].set_xlabel('Time (ms)\n\ntruncated x-limits to show waveform')

Let's try stimulating our fiber. We will use a stimulation amplitude low enough that the membrane voltage appears unaffected.

In [ ]:
ap, time = blockstim.run_sim(-0.5, fiber)

Let's create a function to plot our response to the block waveform, which we'll use going forward. Go ahead and call it now to plot the fiber response.

In [ ]:
def block_plot():
    """Plot the response of a fiber to kHz stimulation."""
    # intracellular stimulation times, specific to the intracellular stimulation we will be adding to the blockstim
    plt.figure(figsize=(10, 5))
    ax = plt.gca()

    ax.plot(blockstim.time, fiber.vm[fiber.loc_index(0.9)], label=r"$V_m(t)$ at 90% fiber length")

    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Transmembrane Potential (mV)')

    ax.axvspan(50, 100, alpha=0.5, color='red', label='block on')

    # label the intracellular stimulation with vertical black lines
    if fiber.stim:
        for istim_time in np.arange(15, 150, 10):
            label = 'Intracellular stim' if istim_time == 15 else '__'
            ax.axvline(x=istim_time, color='black', alpha=0.5, label=label, linestyle='--')

    ax.legend(loc='upper right')
    plt.xlim(-5, 155)


block_plot()
plt.ylim([-82, 20])

## kHz Excitation (below block threshold)
At higher stimulation amplitudes (but still below block threshold), the high frequency stimulation can cause repeated spiking. Let's turn up the stimulus and watch this happen

In [ ]:
ap, time = blockstim.run_sim(-1.5, fiber)  # NOTE: could happen either above or below threshold
block_plot()

## Transmission (with onset response to kHz)
We wanted to stop action potentials, but we are creating a ton of them! Let's try turning up the stimulation. You will see an onset response of spiking caused by the stimulation which fades after some time.

In [ ]:
ap, time = blockstim.run_sim(-2.5, fiber)
block_plot()

We need to know whether we are truly blocking. We have been recording from one end of the fiber (0.9). Let's add spiking activity to the other end of the fiber (0.1) At t=15 ms, the activity turns on and a spike occurs every 10 ms. Note that, as we saw in the plot of our extracellular potentials, stimulation is occurring in the center of the fiber (0.5).
We will stimulate using the same amplitude as before.

In [ ]:
# add intrinsic activity, run with same khz amplitude as above plot
# loc = 0.1 to avoid erroneous end  excitation
fiber.add_intrinsic_activity(loc=0.1, start_time=15, avg_interval=10, num_stims=14)
ap, time = blockstim.run_sim(-2.5, fiber)
block_plot()

This is called "transmission", where the action potentials continue unhindered down the axon.

The action potential takes time to propagate from one end of the fiber to the other, hence the offset between intracellular stimulation pulses and recorded action potentials

## kHz Block
Finally, if we stimulate at an even higher level, we will observe a complete cessation of spike transmission after the block onset response fades.

In [ ]:
ap, time = blockstim.run_sim(-3, fiber)
block_plot()

## Search for kHz block threshold

If we want to determine block threshold, we can use the find_threshold() method. 
This method returns the stimulation amplitude at which the fiber begins to activate, 
and the number of generated action potentials. We can plot the response of the membrane potential, and gating variables threshold stimulation.

This uses the same algorithm as a search for activation threshold, but suprathreshold stimulation is considered the absence, rather than presence of detected action potentials. Thus, intracellular stimulation is key to determining subthreshold stimulation.

The 'block_delay' parameter defines when the function will start looking for action potentials to evaluate whether transmission is occurring. This should be far enough after block turns on to avoid any false positives due to onset response. We will also truncate the end of the simulation so that it ends when block turns off.

In [ ]:
from pyfibers import ThresholdCondition

# Stop simulation when block turns off
blockstim.tstop = 100

# stimamp_top (#) and stimamp_bottom (#) shown to be
# above and below block threshold, respectively, in previous cells
amp, _ = blockstim.find_threshold(
    fiber, condition=ThresholdCondition.BLOCK, stimamp_top=-3, stimamp_bottom=-2, exit_t_shift=None, block_delay=65
)
print(f'Block threshold: {amp} mA')
block_plot()

## Re-excitation
It is important to be careful with the upper bound for your block threshold search, as "re-excitation" can occur at suprathreshold stimulation amplitudes. Thus, it is usually safer to set your initial search bounds subthreshold, to avoid any risk of starting with a top bound that causes re-excitation. 

In [ ]:
blockstim.tstop = 150
ap, time = blockstim.run_sim(-250, fiber)  # NOTE: could happen either above or below threshold
block_plot()